Проект: Исследование опроса клиентов телекомунникацонной компании

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [7]:
query = """
SELECT user.user_id,
       user.lt_day,
       CASE
           WHEN user.lt_day <= 365 THEN 'новый'
           WHEN user.lt_day > 365 THEN 'не новый'
       END AS is_new,
       user.age,
       CASE
           WHEN user.gender_segment == 1 THEN 'Женщина'
           WHEN user.gender_segment == 0 THEN 'Мужчина'
           ELSE 'no_gender'
           END AS gender_segment,
       user.os_name,
       user.cpe_type_name,
       location.country,
       location.city,
       SUBSTR(age_segment.title, 4) AS age_segment,
       SUBSTR(traffic_segment.title, 4) AS traffic_segment,
       SUBSTR(lifetime_segment.title, 4) AS lifetime_segment,
       user.nps_score,
       CASE 
           WHEN user.nps_score >= 9 THEN 'Cторонник'
           WHEN user.nps_score >= 7 THEN 'Нейтрал'
           ELSE 'Критик'
           END AS nps_group
     FROM user
JOIN location ON user.location_id=location.location_id
JOIN age_segment ON user.age_gr_id=age_segment.age_gr_id
JOIN traffic_segment ON user.tr_gr_id=traffic_segment.tr_gr_id
JOIN lifetime_segment ON user.lt_gr_id=lifetime_segment.lt_gr_id;
"""

In [8]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,не новый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Cторонник
1,A001WF,2344,не новый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Cторонник
2,A003Q7,467,не новый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Cторонник


In [9]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Ссылка на презентацию: https://public.tableau.com/views/-2_17146674148250/Story1?:language=en-US&publish=yes&:sid=&:display_count=n&:origin=viz_share_link